# Prompt Engineering using CodeLlamma API


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# Assuming you're using CUDA, set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Available device: {device}')

In [ ]:
# most lightweight model of CodeLlama for instruction prompt
model_id = "codellama/CodeLlama-7b-Instruct-hf"
# model_id = '/Users/guru/research/LLMs/CodeLlama-70-Instruct-hf'
tokenizer = AutoTokenizer.from_pretrained(model_id, force_download=True)
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   torch_dtype=torch.float16,
   device_map="auto",
)

In [ ]:
def generate_chat_response(model, tokenizer, device, chat, max_new_tokens=200):

   inputs = tokenizer.apply_chat_template(chat, return_tensors="pt").to(device)
   # inputs = tokenizer.apply_chat_template(chat, return_tensors="pt")

   output = model.generate(input_ids=inputs, max_new_tokens=max_new_tokens)
   output = output[0].to(device)
   return tokenizer.decode(output)


chat = [
   {"role": "system", "content": "You are a helpful and honest code assistant expert in JavaScript. Please, provide all answers to programming questions in JavaScript"},
   {"role": "user", "content": "Write a function that computes the set of sums of all contiguous sublists of a given list."},
]
response = generate_chat_response(model, tokenizer, device, chat)
print(response)


# RepairLLama

In [ ]:
from datasets import load_dataset

# Load ir1xor1
dataset = load_dataset("ASSERT-KTH/repairllama-datasets", "ir1xor1")
# Load irXxorY
# dataset = load_dataset("ASSERT-KTH/repairllama-dataset", "irXxorY")


In [ ]:
vul = dataset['test'][0]['input']
patch = dataset['test'][0]['output']

prompt = f"Fix the vulnerability in the following code:\n{vul}\n\nPatch:\n{patch}"


chat = [
   {"role": "system", "content": "You are a helpful and honest code assistant expert in Python. Please, provide all answers to programming questions in C"},
   {"role": "user", "content": prompt}]

response = generate_chat_response(model, tokenizer, device, chat, 200)
# print(response)

In [ ]:
# prompt = f"Fix the vulnerability in the following code:\n{vul}\n\nWhat is the patch?"


# chat = [
#    {"role": "system", "content": "You are a helpful and honest code assistant expert in Python. Please, provide all answers to programming questions in C"},
#    {"role": "user", "content": prompt}]

# response = generate_chat_response(model, tokenizer, device, chat, 200)
# print(response)

In [ ]:
# model.half()
# model.to(device)

# response = generate_chat_response(model, tokenizer, device, chat, 500)
# print(response)

In [ ]:
# # load the model
# model = AutoModelForCausalLM.from_pretrained('models/CodeLlama-7b-Instruct-hf')

# response = generate_chat_response(model, tokenizer, device, chat, 500)

# Fine-tuning CodeLLama model

In [ ]:
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    # prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq


In [ ]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
# {'question': 'Name the comptroller for office of prohibition', 'context': 'CREATE TABLE table_22607062_1 (comptroller VARCHAR, ticket___office VARCHAR)', 'answer': 'SELECT comptroller FROM table_22607062_1 WHERE ticket___office = "Prohibition"'}
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

In [ ]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result


In [ ]:
dataset['train'][0]['input']

In [ ]:
def generate_and_tokenize_prompt(data_point):
    full_prompt =f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
What is the patch for the following vulnerability?

### Context:
{dataset['train'][0]['input']}

### Response:
{dataset['train'][0]['output']}
"""
    return tokenize(full_prompt)

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
import torch
import gc


model.train() # put model back into training mode
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)